In [21]:
%pip install pulp

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import pulp

In [23]:
#Declaring constants
NUM_COMPONENTS = 11

In [24]:
# dict for the parameters [height, width] for each component
param_dict = {
    # For square-shaped components, "Component name": [component_length, component_width] 
    # For circular-shaped, "Component name": component_radius
    'chipset':[5,5], 
    'cpu':[3,3], 
    'coolfan':5, 
    'cmos':3, 
    'bios':[3,3], 
    'powercon':[2,1], 
    'usb':[2,1], 
    'ram':[10,2]
}

# define the LP
problem = pulp.LpProblem('motherboard', pulp.LpMinimize)

# define variables
p1 = pulp.LpVariable('p1', lowBound=2, cat='Continuous')
p2 = pulp.LpVariable('p2', lowBound=1, cat='Continuous')
x = pulp.LpVariable.dicts('x',range(1, NUM_COMPONENTS+1), lowBound=0, cat='Continuous')
y = pulp.LpVariable.dicts('y',range(1, NUM_COMPONENTS+1), lowBound=0, cat='Continuous')
W = pulp.LpVariable('W', lowBound=0, cat='Continuous')
H = pulp.LpVariable('H', lowBound=0, cat='Continuous')

# objective function
problem += 2*W + 2*H





In [25]:
# Adding horizontal constraints
problem += x[1] - p1 >= 0
problem += x[1] + p1 + param_dict['chipset'][1] + param_dict['coolfan'] <= x[3]
problem += x[4] - p1 >= 0
problem += x[4] + p1 + 2*param_dict['cmos'] <= x[5]
problem += x[5] + p1 + param_dict['bios'][1] <= x[6]
problem += x[6] + p1 + param_dict['powercon'][1] <= x[7]
problem += x[7] + p1 + param_dict['usb'][1] <= x[8]
problem += x[3] + p1 + param_dict['coolfan'] <= x[8]
problem += x[8] + p2 + param_dict['ram'][1] <= x[9]
problem += x[9] + p2 + param_dict['ram'][1] <= x[10]
problem += x[10] + p2 + param_dict['ram'][1] <= x[11]
problem += x[11] + p2 + param_dict['ram'][1] <= W


In [26]:

# Adding vertical constraints
problem += y[4] - p1 - param_dict['cmos'] >= 0
problem += y[4] + p1 + param_dict['cmos'] <= y[1]
problem += y[1] - (y[4] + param_dict['cmos']) <= d
problem += y[1] + p1 + param_dict['chipset'][0] <= H
problem += y[5] - p1 >= 0
problem += y[5] + p1 + param_dict['bios'][0] <= y[1]
problem += y[6] - p1 >= 0
problem += y[6] + p1 + param_dict['powercon'][0] + param_dict['coolfan'] <= y[3]
problem += y[3] + p1 + param_dict['coolfan'] <= H
problem += y[7] - p1 >= 0
problem += y[7] + p1 + param_dict['usb'][0] + param_dict['coolfan'] <= y[3]
problem += y[8] - p1 >= 0
problem += y[8] + param_dict['ram'][0] <= H
problem += y[9] - p1 >= 0
problem += y[9] + param_dict['ram'][0] <= H
problem += y[10] - p1 >= 0
problem += y[10] + param_dict['ram'][0] <= H
problem += y[11] - p1 >= 0
problem += y[11] + param_dict['ram'][0] <= H


In [27]:

#alignment constraints
problem += x[2] + 0.5*param_dict['cpu'][1] == x[3]
problem += x[3] + param_dict['coolfan'] == x[6]
problem += y[2] + 0.5*param_dict['cpu'][0] == y[3]


In [28]:
print(problem)
# solve
problem.solve(pulp.PULP_CBC_CMD(msg=0))

# create dictionary with the solutions
solution = {}
for var in problem.variables():
    solution[var.name] = var.varValue

print(solution)
print("Optimised Perimeter of motherboard: ", 2*(solution["W"] + solution["H"]), "cm")
print("Width of motherboard:", solution['W'],  "Height of motherboard:", solution["H"])
print(f"Coordinates of Chipset (component 1): ",(solution["x_1"] , solution["y_1"]))
print(f"Coordinates of CPU (component 2): ",(solution["x_2"] , solution["y_2"]))
print(f"Coordinates of Cooling Fan (component 3): ",(solution["x_3"] , solution["y_3"]))
print(f"Coordinates of CMOS (component 4): ",(solution["x_4"] , solution["y_4"]))
print(f"Coordinates of BIOS (component 5): ",(solution["x_5"] , solution["y_5"]))
print(f"Coordinates of Power Converter (component 6): ",(solution["x_6"] , solution["y_6"]))
print(f"Coordinates of USB Converter (component 7): ",(solution["x_7"] , solution["y_7"]))
print(f"Coordinates of RAM Memory Stick 1 (component 8): ",(solution["x_8"] , solution["y_8"]))
print(f"Coordinates of RAM Memory Stick 2 (component 9): ",(solution["x_9"] , solution["y_9"]))
print(f"Coordinates of RAM Memory Stick 3 (component 10): ",(solution["x_10"] , solution["y_10"]))
print(f"Coordinates of RAM Memory Stick 4 (component 11): ",(solution["x_11"] , solution["y_11"]))

motherboard:
MINIMIZE
2*H + 2*W + 0
SUBJECT TO
_C1: - p1 + x_1 >= 0

_C2: p1 + x_1 - x_3 <= -10

_C3: - p1 + x_4 >= 0

_C4: p1 + x_4 - x_5 <= -6

_C5: p1 + x_5 - x_6 <= -3

_C6: p1 + x_6 - x_7 <= -1

_C7: p1 + x_7 - x_8 <= -1

_C8: p1 + x_3 - x_8 <= -5

_C9: p2 + x_8 - x_9 <= -2

_C10: p2 - x_10 + x_9 <= -2

_C11: p2 + x_10 - x_11 <= -2

_C12: - W + p2 + x_11 <= -2

_C13: - p1 + y_4 >= 3

_C14: p1 - y_1 + y_4 <= -3

_C15: - H + p1 + y_1 <= -5

_C16: - p1 + y_5 >= 0

_C17: p1 - y_1 + y_5 <= -3

_C18: - p1 + y_6 >= 0

_C19: p1 - y_3 + y_6 <= -7

_C20: - H + p1 + y_3 <= -5

_C21: - p1 + y_7 >= 0

_C22: p1 - y_3 + y_7 <= -7

_C23: - p1 + y_8 >= 0

_C24: - H + y_8 <= -10

_C25: - p1 + y_9 >= 0

_C26: - H + y_9 <= -10

_C27: - p1 + y_10 >= 0

_C28: - H + y_10 <= -10

_C29: - p1 + y_11 >= 0

_C30: - H + y_11 <= -10

_C31: x_2 - x_3 = -1.5

_C32: x_3 - x_6 = -5

_C33: y_2 - y_3 = -1.5

VARIABLES
H Continuous
W Continuous
2 <= p1 Continuous
1 <= p2 Continuous
x_1 Continuous
x_10 Continuous
x_11